In [1]:
import pandas as pd
import re
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Embedding, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import nltk
import warnings
warnings.filterwarnings("ignore")


In [2]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [3]:
file_path = "train_chat.csv"
df=pd.read_csv('train_chat.csv')
df

text
0    <HUMAN>: What is a panic attack?\n<ASSISTANT>:...
1    <HUMAN>: What are symptoms of panic attack vs....
2    <HUMAN>: What are the types of Mental Illness?...
3    <HUMAN>: What does mental-illness mean?\n<ASSI...
4    <HUMAN>: How can you treat mental illness?\n<A...
..                                                 ...
167  <HUMAN>: Who are you?\n<ASSISTANT>: Hello ther...
168  <HUMAN>: What is the helpline number for suici...
169  <HUMAN>: Which is the best mental health hospi...
170  <HUMAN>: Who is the best hypnotherapist in Lon...
171  <HUMAN>: Who is the best psychiatrist in Mumba...

[172 rows x 1 columns]

In [4]:
# Function to extract questions and responses
def extract_qna(text):
    match = re.match(r"<HUMAN>:\s*(.*?)\n<ASSISTANT>:\s*(.*)", text, re.DOTALL)
    return match.groups() if match else (None, None)

df[['question', 'response']] = df['text'].apply(lambda x: pd.Series(extract_qna(x)))
df = df.drop(columns=['text'])


In [5]:
# Text Preprocessing
def preprocess_text(text):
    stemmer = PorterStemmer()
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text.lower())
    words = [stemmer.stem(word) for word in words if word.isalnum() and word not in stop_words]
    return ' '.join(words)

df['question'] = df['question'].apply(preprocess_text)

df.dropna(inplace=True)

In [6]:
# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['question'])
X = tokenizer.texts_to_sequences(df['question'])
X = pad_sequences(X, padding='post')

In [7]:
# Encode responses
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df["response"])

In [8]:
# Deep Learning Model with LSTM
model = Sequential([
    Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=X.shape[1]),
    Bidirectional(LSTM(128, return_sequences=True)),
    Bidirectional(LSTM(64)),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(len(np.unique(y)), activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [9]:
# Train Model
X_train = np.array(X)
y_train = np.array(y)
model.fit(X_train, y_train, epochs=20, batch_size=8, verbose=1)

Epoch 1/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 17s 23ms/step - accuracy: 0.0000e+00 - loss: 5.1565
Epoch 2/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.0000e+00 - loss: 5.1497
Epoch 3/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.0118 - loss: 5.1475
Epoch 4/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.0055 - loss: 5.1420
Epoch 5/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.0206 - loss: 5.1363
Epoch 6/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.0070 - loss: 5.0819
Epoch 7/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.0149 - loss: 4.9956
Epoch 8/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.0547 - loss: 4.8166
Epoch 9/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.0541 - loss: 4.5708
Epoch 10/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.0127 - loss: 4.4725
Epoch 11/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.0561 - loss: 4.1619
Epoch 12/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - 

In [10]:
def chatbot_response(user_question):
    user_question = preprocess_text(user_question)
    user_vector = tokenizer.texts_to_sequences([user_question])
    user_vector = pad_sequences(user_vector, maxlen=X.shape[1], padding='post')
    prediction = model.predict(user_vector)
    response_index = np.argmax(prediction)
    return label_encoder.inverse_transform([response_index])[0]


In [ ]:
#Interactive Chatbot
print("Chatbot is ready! Type 'exit' to stop.")
while True:
    user_input = input("What are the types of Mental Illness")
    if user_input.lower() == 'exit':
        print("Chatbot: Goodbye!")
        break
    response = chatbot_response(user_input)
    print(f"Chatbot: {response}")


Chatbot is ready! Type 'exit' to stop.


Post-traumatic stress disorder (PTSD)  


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Chatbot: Hello there! I am your empathetic Mental Health conversational chatbot, designed to provide personalized suggestions and support. My purpose is to lend a listening ear, offer guidance, and assist you in navigating through any mental health challenges you might be facing. Feel free to share your thoughts and concerns, and I'm here to help you to the best of my abilities. Remember, you are not alone, and I'm here to support you throughout this journey.


Post-traumatic stress disorder (PTSD)  What are the types of Mental Illness


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
Chatbot: There are many different conditions that are recognized as mental illnesses. The more common types include:

1. Anxiety disorders: People with anxiety disorders respond to certain objects or situations with fear and dread, as well as with physical signs of anxiety or panic, such as a rapid heartbeat and sweating. An anxiety disorder is diagnosed if the person's response is not appropriate for the situation, if the person cannot control the response, or if the anxiety interferes with normal functioning. 

2. Mood disorders: These disorders, also called affective disorders, involve persistent feelings of sadness or periods of feeling overly happy, or fluctuations from extreme happiness to extreme sadness.

3. Psychotic disorders: Psychotic disorders involve distorted awareness and thinking. Two of the most common symptoms of psychotic disorders are hallucinations and delusions. Schizophrenia is an example of a psychotic disorder.

4. Eating 

Post-traumatic stress disorder (PTSD)   How can you treat mental illness


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
Chatbot: Mental illness is treated by a variety of healthcare professionals who specialize in mental health and emotional well-being. Here are some of the key professionals who play a vital role in treating mental illness:

1. Primary care doctor: These doctors are either MDs or DOs who have general knowledge in treating a wide variety of common conditions. They are sometimes called general practitioners.

2. Physician assistant (PA): These caregivers are not doctors, but they are trained to identify symptoms of mental illness and to treat mental disorders under a doctor’s supervision.

3. Nurse practitioner: These registered nurses (RNs) have extra training, including some background in treating psychiatric problems.

4. Psychiatrist: These are medical doctors (MDs) who specialize in the diagnosis and treatment of mental illnesses. Psychiatrists can prescribe drugs as part of their treatment plan. They are also trained in psychotherapy, a form of 